In [6]:
import numpy as np
import os
import open3d as o3d
from glob import glob
import time
import cv2

# --- New Imports for Jupyter Animation ---
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# --- 1. Original Camera Intrinsics (1024x1280) ---
K_matrix_orig = np.array([
    [935.2355857804463, 0, 656.1572332633887],
    [0, 935.7905325732659, 513.7144019593092],
    [0, 0, 1]
])
fx_orig = K_matrix_orig[0, 0]
fy_orig = K_matrix_orig[1, 1]
cx_orig = K_matrix_orig[0, 2]
cy_orig = K_matrix_orig[1, 2]

# --- 2. Define Geometric Processing Parameters ---
H_orig, W_orig = 1024, 1280
CROP_Y_START = 160 # 40 on 256
CROP_Y_END = 900   # 225 on 256
H_crop = CROP_Y_END - CROP_Y_START # 740
W_square = 740
H_square = 740
X_CROP_START = (W_orig - W_square) // 2 # 270
H_final = 256
W_final = 256

# --- 3. Calculate NEW K Matrix (Unused in this script, but correct) ---
cx_1 = cx_orig - X_CROP_START
cy_1 = cy_orig - CROP_Y_START
scale = H_final / H_square
fx_new = fx_orig * scale
fy_new = fy_orig * scale
cx_new = cx_1 * scale
cy_new = cy_1 * scale

# --- 4. Helper Function to Process Image ---
def process_image(img_full):
    img_crop1 = img_full[CROP_Y_START:CROP_Y_END, :] # 740x1280
    img_crop2 = img_crop1[:, X_CROP_START:X_CROP_START + W_square] # 740x740
    img_final = cv2.resize(img_crop2, (W_final, H_final), interpolation=cv2.INTER_NEAREST) # 256x256
    return img_final

# --- 5. Helper Function to Visualize Depth Map ---
def visualize_depth(depth_map_processed, max_depth=30.0):
    depth_visual = np.clip(depth_map_processed, 0.1, max_depth)
    depth_visual = (depth_visual - 0.1) / (max_depth - 0.1)
    depth_visual_u8 = (depth_visual * 255).astype(np.uint8)
    # Apply colormap and convert to RGB for matplotlib
    depth_color_bgr = cv2.applyColorMap(depth_visual_u8, cv2.COLORMAP_INFERNO)
    return cv2.cvtColor(depth_color_bgr, cv2.COLOR_BGR2RGB)

# --- Project to 3D Points using BOTH K matrices ---
def project_depth_to_3d(depth, K, H, W):
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    xx, yy = np.meshgrid(np.arange(W), np.arange(H))
    z = depth
    x = (xx - cx) * z / fx
    y = (yy - cy) * z / fy
    return x, y, z

# Use WRONG K: original for 1024x1280 resized to 256x256
K_wrong = K_matrix_orig.copy()
K_wrong[0, 0] *= (W_final / W_orig)
K_wrong[1, 1] *= (H_final / H_orig)
K_wrong[0, 2] *= (W_final / W_orig)
K_wrong[1, 2] *= (H_final / H_orig)

# Use CORRECT K (from your crop/resize math)
K_correct = np.array([
    [fx_new, 0, cx_new],
    [0, fy_new, cy_new],
    [0, 0, 1]
])
# --- 6. Load Your .npy File Sequence ---
sequence_folder = '/mnt/sbackup/Server_3/harshr/m2p2_data_bags/burke_lake_bags/Dataset/depth_LR_2024-09-02_22-24-34_chunk0014_processed'
all_depth_files = glob(os.path.join(sequence_folder, "*.npy"))
all_depth_files.sort(key=lambda f: int(os.path.basename(f).split('.')[0]))
print(f"\nFound {len(all_depth_files)} files to animate...")

# --- 7. Set up the Matplotlib Figure for Animation ---
fig, ax = plt.subplots(figsize=(8, 8))
print("Starting animation...")

# --- 8. Run the Animation Loop ---
try:
    for i, filepath in enumerate(all_depth_files):
        
        # --- Process Data ---
        depth_img_full = np.load(filepath)
        depth_processed = process_image(depth_img_full)
        depth_color = visualize_depth(depth_processed)
        
        # --- Update Visualizer (Matplotlib) ---
        clear_output(wait=True) # Clear the previous image
        ax.clear()
        # ax.imshow(depth_color)
        # ax.set_title(f'Processed Depth Map (256x256) - Frame {i}')
        # ax.axis('off')
        # display(fig) # Display the updated figure


        

        xw, yw, zw = project_depth_to_3d(depth_processed, K_wrong, H_final, W_final)
        xc, yc, zc = project_depth_to_3d(depth_processed, K_correct, H_final, W_final)

        # --- Plot Bird's-Eye View (XZ plane) ---
        ax2 = fig.add_subplot(1, 2, 2)
        ax2.clear()
        ax2.scatter(xw.ravel(), zw.ravel(), color='red', alpha=0.2, label='Wrong K')
        ax2.scatter(xc.ravel(), zc.ravel(), color='green', alpha=0.2, label='Correct K')
        ax2.legend()
        ax2.set_title('3D Scan Projection (XZ plane)')
        ax2.set_xlabel('X (meters)')
        ax2.set_ylabel('Z (meters)')
        ax2.set_xlim(-5, 5)
        ax2.set_ylim(0, 10)

        
        time.sleep(0.05) # Control playback speed

except KeyboardInterrupt:
    print("Animation stopped.")
finally:
    plt.close(fig) # Close the figure when done


Animation stopped.
